<a href="https://colab.research.google.com/github/victory1217/HHA504_SQLtrigger.function/blob/main/SQLtable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Step 1 - Import any needed packages

In [10]:
import pandas as pd 
import sqlalchemy 
from sqlalchemy import create_engine
!pip install pymysql 

##Step 2 - Manually create quick table in Google Collab 

In [11]:
proceduregrouper = pd.DataFrame ({'PatientID' : ['1111', '2222', '3333', '4444', '5555', '6666', '7777', '8888'], 'LastName' : ['Roberts', 'Gonzalez', 'McMillin', 'Silva', 'Anthony', 'Washington', 'Martinez', 'Rodriguez'], 'FirstName' : ['Veronica', 'Jacqueline', 'Miranda', 'Joshua', 'Xavier', 'Mike', 'Jennifer', 'Allen'], 
                             'SystolicBloodPressure':['121', '120.5', '140', '131', '138.7', '119', '142', '118'], 'ProcedureCode': ['169553002', '117015009', '301807007', '430193006', '288086009', '76601001', '5880005', '11466000'], 
                             'ProcedureCost' : ['14896.56', '2070.44', '12914.35', '416.69', '11354.55', '3001.57', '516.65', '612.34']})
proceduregrouper

,PatientID,LastName,FirstName,SystolicBloodPressure,ProcedureCode,ProcedureCost
0,1111,Roberts,Veronica,121,169553002,14896.56
1,2222,Gonzalez,Jacqueline,120.5,117015009,2070.44
2,3333,McMillin,Miranda,140,301807007,12914.35
3,4444,Silva,Joshua,131,430193006,416.69
4,5555,Anthony,Xavier,138.7,288086009,11354.55
5,6666,Washington,Mike,119,76601001,3001.57
6,7777,Martinez,Jennifer,142,5880005,516.65
7,8888,Rodriguez,Allen,118,11466000,612.34


##Step 3 - Connect to SQL and forward quick table to selected schema 
###Note: Removed personal login information for security purposes 

In [12]:
MYSQL_HOSTNAME = '20.62.193.224'
MYSQL_USER = 'INSERT HERE'
MYSQL_PASSWORD = 'INSERT HERE'
MYSQL_DATABASE = 'ahi'

connection_string = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOSTNAME}:3305/{MYSQL_DATABASE}'
engine = create_engine(connection_string)

TABLENAME = MYSQL_USER + 'proceduregrouper'

proceduregrouper.to_sql(TABLENAME, con=engine)

##Step 4 - Create Trigger within newly added table in schema 
###Note: The code below provides an error but does work (since the new table in schema now has a "Triggers" tab) 

In [ ]:
DELIMITER $$
CREATE TRIGGER systolic BEFORE INSERT ON userproceduregrouper
FOR EACH ROW 
BEGIN
    IF NEW.SystolicBloodPressure >= 120 THEN 
    SIGNAL SQLSTATE '45000'
    SET MESSAGE_TEXT = 'ERROR: Systolic blood pressure must be 120 mm Hg or below for procedure!';
    END IF; 
END; $$

##Step 5 - Create Function within newly added table in schema 

In [ ]:
DELIMITER $$
CREATE FUNCTION ProcedureCost(
    cost DECIMAL(10,2)
)

RETURNS VARCHAR(20)
DETERMINISTIC
BEGIN
DECLARE procedureCost VARCHAR(20);
IF cost > 10000 THEN
SET procedureCost = ‘expensive’;

ELSEIF (cost >= 1000 AND
credit <= 5000) THEN
SET procedureCost = 'standard';

ELSEIF cost < 1000 THEN
SET procedureCost = 'cheap';
END IF;
-- return the procedure cost category
RETURN (procedureCost);
END$$
DELIMITER ;